### Scrape Senate candidate face urls from ballotpedia

Note: The output file will be different using the updated wmpcand_101422_wmpid.csv file

In [1]:
import pandas as pd
import bs4 as bs # pulling data out of HTML and XML files.
import urllib.request # opening and reading URLs
import re # pattern matching

In [26]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/home/jyao01/.local/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#Download the website's html
source = urllib.request.urlopen("https://ballotpedia.org/United_States_Senate_elections,_2022").read()

In [3]:
#Parse the html
soup = bs.BeautifulSoup(source)

In [4]:
urls_css = "small center a"

In [5]:
urls = soup.select(urls_css)

In [6]:
extracted_urls = [url["href"] for url in urls]

In [7]:
full_urls = ["https://ballotpedia.org" + url_suffix for url_suffix in extracted_urls]

In [8]:
len(full_urls)

36

In [9]:
full_urls

['https://ballotpedia.org/United_States_Senate_election_in_Alabama,_2022',
 'https://ballotpedia.org/United_States_Senate_election_in_Alaska,_2022',
 'https://ballotpedia.org/United_States_Senate_election_in_Arizona,_2022',
 'https://ballotpedia.org/United_States_Senate_election_in_Arkansas,_2022',
 'https://ballotpedia.org/United_States_Senate_election_in_California,_2022',
 'https://ballotpedia.org/United_States_Senate_special_election_in_California,_2022',
 'https://ballotpedia.org/United_States_Senate_election_in_Colorado,_2022',
 'https://ballotpedia.org/United_States_Senate_election_in_Connecticut,_2022',
 'https://ballotpedia.org/United_States_Senate_election_in_Florida,_2022',
 'https://ballotpedia.org/United_States_Senate_election_in_Georgia,_2022',
 'https://ballotpedia.org/United_States_Senate_election_in_Hawaii,_2022',
 'https://ballotpedia.org/United_States_Senate_election_in_Idaho,_2022',
 'https://ballotpedia.org/United_States_Senate_election_in_Illinois,_2022',
 'https:

Now that we've got all the links, we need to visit each, extract what we need and save it somewhere.

In [10]:
#Now do this in a loop, for the first 5 urls
bp_urls = []
bp_names = []

In [11]:
for i in full_urls[:]:
  source = urllib.request.urlopen(i).read()
  soup = bs.BeautifulSoup(source)
  img_css = "img.image-candidate-thumbnail, img[id=placeholder_image]"
  img = soup.select(img_css)
  img = img[0:5]
  bp_urls.extend(list([i['src'] for i in img]))
  n_css = "td.votebox-results-cell--text"
  nm = soup.select(n_css)
  nm = nm[0:5]
  bp_names.extend(list([url.text for url in nm]))

In [12]:
df = pd.DataFrame(columns=('bp_name_raw', 'bp_url'))

df['bp_name_raw']=bp_names

df['bp_url']=bp_urls

In [13]:
df.tail()

,bp_name_raw,bp_url
175,Ronald Harold Johnson (R) \n\t\t\t\t\t\t\t\t\...,https://s3.amazonaws.com/ballotpedia-api4/file...
176,Mandela Barnes (D),https://s3.amazonaws.com/ballotpedia-api4/file...
177,Adam Nicholas Paul (Logic Party) (Write-in) \...,https://cdn.ballotpedia.org/images/thumb/f/fb/...
178,Scott Aubart (American Independent Party) (Wri...,https://cdn.ballotpedia.org/images/thumb/f/fb/...
179,Other/Write-in votes,https://s3.amazonaws.com/ballotpedia-api4/file...


In [14]:
df2 = df['bp_name_raw'].str.rsplit("(", n=1, expand=True)

In [15]:
df2 = df2.rename(columns={0:'name'})

In [16]:
df3 =df2['name'].str.rsplit("(", n=1, expand=True)
df3 = df3.rename(columns={0:'name'})

In [17]:
df3['bp_name'] = df3['name'].str.replace("\n", "").replace("\t", "")

In [18]:
df3['bp_name']

0                Katie Britt 
1                  Will Boyd 
2            John Sophocleus 
3                 Katie Britt
4                   Mo Brooks
                ...          
175    Ronald Harold Johnson 
176           Mandela Barnes 
177       Adam Nicholas Paul 
178             Scott Aubart 
179      Other/Write-in votes
Name: bp_name, Length: 180, dtype: object

In [19]:
df['bp_name']=df3['bp_name']

In [20]:
df['bp_url_filename'] = df['bp_url'].str.replace("https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/100/100/", '')

In [21]:
df['bp_name'] = df['bp_name'].str.strip()

In [22]:
df = df.drop_duplicates(subset=['bp_name'], keep='first')

In [23]:
df.tail(10)

,bp_name_raw,bp_url,bp_name,bp_url_filename
166,Gerald Malloy (R),https://s3.amazonaws.com/ballotpedia-api4/file...,Gerald Malloy,Gerald_Malloy.jpeg
167,Dawn Ellis (Independent) \n\t\t\t\t\t\t\t\t\t...,https://s3.amazonaws.com/ballotpedia-api4/file...,Dawn Ellis,DawnEllis.jpg
168,Natasha Diamondstone-Kohout (Green Mountain Pe...,https://cdn.ballotpedia.org/images/thumb/f/fb/...,Natasha Diamondstone-Kohout,https://cdn.ballotpedia.org/images/thumb/f/fb/...
169,Kerry Patrick Raheb (Independent),https://s3.amazonaws.com/ballotpedia-api4/file...,Kerry Patrick Raheb,KerryRahebVT.png
170,Patty Murray (D) \n\t\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Patty Murray,Patty_Murray.jpg
171,Tiffany Smiley (R) \n\t\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Tiffany Smiley,TiffanySmiley.jpeg
175,Ronald Harold Johnson (R) \n\t\t\t\t\t\t\t\t\...,https://s3.amazonaws.com/ballotpedia-api4/file...,Ronald Harold Johnson,Ron_Johnson.jpg
176,Mandela Barnes (D),https://s3.amazonaws.com/ballotpedia-api4/file...,Mandela Barnes,Mandela-Barnes.jpg
177,Adam Nicholas Paul (Logic Party) (Write-in) \...,https://cdn.ballotpedia.org/images/thumb/f/fb/...,Adam Nicholas Paul,https://cdn.ballotpedia.org/images/thumb/f/fb/...
178,Scott Aubart (American Independent Party) (Wri...,https://cdn.ballotpedia.org/images/thumb/f/fb/...,Scott Aubart,https://cdn.ballotpedia.org/images/thumb/f/fb/...


### Load the masterfile

In [30]:
s = pd.read_csv("../datasets/candidates/wmpcand_101422_wmpid.csv")

In [31]:
s.columns

Index(['wmpid', 'genelect_cd', 'CurrCand', 'cand_name', 'cand_id',
       'cand_office', 'cand_office_st', 'cand_office_dist',
       'cand_party_affiliation', 'cand_incumbent_challenger_open_s',
       'dateadded_cd', 'office_wapo', 'result_wapo', 'primarydate_wapo',
       'trumpe_wapo', 'gender_wmp', 'gender_crp', 'latino_wmp', 'latino_crp',
       'race_wmp', 'race_crp1', 'race_crp2', 'race_crpmena', 'full_name',
       'first_name', 'last_name'],
      dtype='object')

In [32]:
s.genelect_cd.value_counts(dropna=False)

0    2828
1    1468
Name: genelect_cd, dtype: int64

In [33]:
s = s.loc[(s['genelect_cd']==1)&(s['cand_office']=='S')]

In [34]:
s.shape

(172, 26)

In [35]:
s.head()

,wmpid,genelect_cd,CurrCand,cand_name,cand_id,cand_office,cand_office_st,cand_office_dist,cand_party_affiliation,cand_incumbent_challenger_open_s,...,gender_crp,latino_wmp,latino_crp,race_wmp,race_crp1,race_crp2,race_crpmena,full_name,first_name,last_name
3614,WMPID1299,1,Y,"BOOZMAN, SEN. JOHN",S0AR00150,S,AR,0.0,REP,INCUMBENT,...,M,NaN,NaN,White,W,NaN,NaN,John Boozman,John,Boozman
3616,WMPID1302,1,Y,"KELLY, MARK",S0AZ00350,S,AZ,0.0,DEM,INCUMBENT,...,M,0.0,N,White,W,NaN,N,Mark Kelly,Kelly,Mark
3619,WMPID1305,1,Y,"BENNET, MICHAEL F.",S0CO00211,S,CO,0.0,DEM,INCUMBENT,...,M,0.0,N,White,W,NaN,N,Michael Bennet,Michael,Bennet
3620,WMPID1315,1,Y,"BLUMENTHAL, RICHARD",S0CT00177,S,CT,0.0,DEM,INCUMBENT,...,M,NaN,NaN,White,W,NaN,NaN,Richard Blumenthal,Richard,Blumenthal
3621,WMPID1321,1,Y,"RUBIO, MARCO",S0FL00338,S,FL,0.0,REP,INCUMBENT,...,M,1.0,Y,Hispanic/Latino,H,NaN,NaN,Marco Rubio,Marco,Rubio


In [36]:
s = s[['wmpid', 'cand_name', 'full_name', 'cand_office_st', 'cand_party_affiliation']]

In [37]:
s.head()

,wmpid,cand_name,full_name,cand_office_st,cand_party_affiliation
3614,WMPID1299,"BOOZMAN, SEN. JOHN",John Boozman,AR,REP
3616,WMPID1302,"KELLY, MARK",Mark Kelly,AZ,DEM
3619,WMPID1305,"BENNET, MICHAEL F.",Michael Bennet,CO,DEM
3620,WMPID1315,"BLUMENTHAL, RICHARD",Richard Blumenthal,CT,DEM
3621,WMPID1321,"RUBIO, MARCO",Marco Rubio,FL,REP


In [38]:
len(s['wmpid'].unique())

172

In [39]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [40]:
dat = fuzzy_merge(s, df, 'full_name', 'bp_name', threshold=90)

In [41]:
dat

,wmpid,cand_name,full_name,cand_office_st,cand_party_affiliation,matches
3614,WMPID1299,"BOOZMAN, SEN. JOHN",John Boozman,AR,REP,John Boozman
3616,WMPID1302,"KELLY, MARK",Mark Kelly,AZ,DEM,Mark Kelly
3619,WMPID1305,"BENNET, MICHAEL F.",Michael Bennet,CO,DEM,Michael Bennet
3620,WMPID1315,"BLUMENTHAL, RICHARD",Richard Blumenthal,CT,DEM,Richard Blumenthal
3621,WMPID1321,"RUBIO, MARCO",Marco Rubio,FL,REP,Marco Rubio
...,...,...,...,...,...,...
4276,WMPID1518,"MURPHY, ROBERT T",Robert T. Murphy,OK,LIB,Robert Murphy
4277,WMPID1519,"WYDEN, RONALD LEE",Ron Wyden,OR,DEM,Ron Wyden
4279,WMPID2065,"FETTERMAN, JOHN KARL",John Karl Fetterman,PA,DEM,John Fetterman
4284,WMPID1531,"CRAPO, MICHAEL D",Mike Crapo,ID,REP,Mike Crapo


In [42]:
dat.loc[4288, 'matches']

''

In [43]:
dat.loc[dat['matches']=='']

,wmpid,cand_name,full_name,cand_office_st,cand_party_affiliation,matches
3622,WMPID174,"KNEPPER, HOWARD",Howard Knepper,FL,W,
3635,WMPID1370,"JOHN, XAN MR.",Xan John,LA,OTH,
3651,WMPID1450,"JOHNSON, RON HAROLD MR.",Ron Johnson,WI,REP,
3679,WMPID2592,"SANDERS, JARMAL JABBAR REV",Jarmal Jabbar Sander,AL,UN,
3682,WMPID1966,"BOWERS, RICHARD ADAM MR.",Richard Adam Bowers,AL,NPA,
...,...,...,...,...,...,...
4218,WMPID3887,"D'AMICO, SAMUEL NICHOLAS MR",Samuel Nicholas D?Amico,WI,DEM,
4247,WMPID1481,"KENNEDY, JOHN NEELY",John Kennedy,LA,REP,
4248,WMPID4986,"MCMORRIS, BRANNON L",Brannon Mcmorris,LA,IND,
4274,WMPID1514,"HASSAN, MARGARET WOOD",Margaret Hassan,NH,DEM,


In [44]:
sen = dat.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [45]:
sen.head()

,wmpid,cand_name,full_name,cand_office_st,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID1299,"BOOZMAN, SEN. JOHN",John Boozman,AR,REP,John Boozman,John Boozman (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,John Boozman,johnboozmanupdated.jpg,both
1,WMPID1302,"KELLY, MARK",Mark Kelly,AZ,DEM,Mark Kelly,Mark Kelly (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Mark Kelly,Mark-Kelly.png,both
2,WMPID1305,"BENNET, MICHAEL F.",Michael Bennet,CO,DEM,Michael Bennet,Michael Bennet (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Michael Bennet,Michael_Bennet.jpg,both
3,WMPID1315,"BLUMENTHAL, RICHARD",Richard Blumenthal,CT,DEM,Richard Blumenthal,Richard Blumenthal (D / Working Families Party...,https://s3.amazonaws.com/ballotpedia-api4/file...,Richard Blumenthal,Richard_Blumenthal.jpg,both
4,WMPID1321,"RUBIO, MARCO",Marco Rubio,FL,REP,Marco Rubio,Marco Rubio (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Marco Rubio,Marco_Rubio.jpg,both


In [46]:
sen._merge.value_counts()

both          92
left_only     80
right_only     0
Name: _merge, dtype: int64

In [48]:
sen.to_csv("./data/bp2022_sen_scraped_face.csv", index=False)